Check Create_Own_pkl_file.ipynb file to get the pkl file used here.

In [72]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity


def reduce_memory(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

games = reduce_memory(pd.read_csv('./final_dataset.csv'))
n_recommendation = 20
vectors = pickle.load(open("./vectors_final.pkl",'rb'))

# similarity=pickle.load(open("./similarity_forDesc.pkl",'rb'))

# def recommend(game):
#     index = games[games['title'] == game].index[0]
#     sim_scores = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
#     print(sim_scores)
#     game_lists=[]
#     for i in sim_scores[1:n_recommendation]:
#         game_lists.append(games.iloc[i[0]].title)
#     return game_lists
def recommend(game):
    index = games[games['title'] == game].index[0]

    item_vector = vectors[index]
    similarities = cosine_similarity(item_vector, vectors).flatten()
    recommended_indices = similarities.argsort()[::-1]  # Get the indices of the most similar items
    game_lists=[]
    for i in recommended_indices[1:n_recommendation]:
        game_lists.append(games.iloc[i].title)
    return (game_lists)

# recommend('God Sword')

In [18]:
recommend("Call of Duty®: Black Ops Cold War")

[0.18795299 0.29029426 0.32891773 ... 0.04082653 0.08461538 0.04482858]
[43458 43770 43459 ... 20000 19980 19987]
[1.         0.61782654 0.52787967 ... 0.         0.         0.        ]
Call of Duty®: Modern Warfare®
Call of Duty®: Vanguard
The Undisputables : Online Multiplayer Shooter
Tactical Operations Force
Call of Duty®: Modern Warfare® II
Cyberfrags '69
Burst Squad
Object N
Men of War: Assault Squad 2 - Cold War
World Of Stalkers
World of Warplanes
Warplanes: Battles over Pacific
Men of War: Vietnam
Modern Assault Tanks
The Last War
Battlefield 4™
Battle Strike World War
Poly Squad
PIRATECRAFT


In [75]:
recommend("Call of Duty® 2")

['Call of Duty®',
 'Call of Duty: United Offensive',
 'Call of Duty®: Modern Warfare® Remastered (2017)',
 'Warrior Paint - 2005 GOTY Edition',
 'Verdun',
 'Red Orchestra: Ostfront 41-45',
 'Fog Of War - Free Edition',
 'Day of Defeat: Source',
 'Battlefield: Bad Company 2 Vietnam',
 'Day of Infamy',
 'Battle For The Sun',
 'Red Orchestra 2: Heroes of Stalingrad with Rising Storm',
 'Operation Flashpoint: Dragon Rising',
 'Call of Duty® 4: Modern Warfare® (2007)',
 'Toy Soldiers',
 'World of Warplanes',
 'Battle Islands',
 'Medal of Honor™',
 'Land of War - The Beginning']

In [42]:
def get_game_recommendations(game_name):
    n=10
    game_similarity_df = pickle.load(open("./game_similarity_df.pkl",'rb'))
    mapped_df = pickle.load(open("./mapped_df.pkl",'rb'))
    # Check if the game name exists in the filtered_df_player_count
    if game_name not in mapped_df['title'].values:
        print("The game lacks rating. Unable to find recommendations.")
        return

    
    # Get the app_id of the input game
    app_id = mapped_df.loc[mapped_df['title'] == game_name, 'app_id'].values[0]
    
    # Check if the app_id is in the similarity matrix
    if app_id not in game_similarity_df.index:
        print("No similar games found/ Low rated game ")
        return
    
    # Get similarity scores for the game and sort them in descending order
    similarity_scores = game_similarity_df.loc[app_id].sort_values(ascending=False)
    
    # Get top n similar app_ids (excluding the game itself)
    top_app_ids = similarity_scores.iloc[1:n+1].index  # Exclude the first as it's the game itself
    
    # Map app_ids to game titles
    recommendations = mapped_df[mapped_df['app_id'].isin(top_app_ids)]['title'].tolist()
    
    return (recommendations)

In [69]:
ground_truth = {
    "Call of Duty® 2": ["Call of Duty®", "Call of Duty®: Modern Warfare® Remastered (2017)", 
                        "Battlefield: Bad Company 2 Vietnam", "Call of Duty: United Offensive","Verdun"],
    "Half-Life 2: Episode One": ["Half-Life 2","Half-Life 2: Episode Two", "Half-Life: Source","Half-Life 2: Update"],  # Relevant items for Half-Life
    "Battlefield: Bad Company™ 2": ["Battlefield 4™","Battlefield™ 2042","World War 3","Vanguard: Normandy 1944","Battlefield: Bad Company 2 Vietnam"]
}

In [70]:
def calculate_precision_recall(input_title, recommended_titles, ground_truth, top_n=3):
    """
    Calculate precision, recall, and other metrics for a single input title.
    """
    # Get the relevant items from the ground truth for this test item
    relevant_items = ground_truth.get(input_title, [])
    if not relevant_items:
        return {"precision": 0, "recall": 0, "true_positives": 0, "false_positives": 0, "false_negatives": 0}
    # Consider only the top N recommended items
    top_recommended = recommended_titles[:top_n]
    # Calculate true positives, false positives, and false negatives
    true_positives = len(set(relevant_items) & set(top_recommended))
    false_positives = len(set(top_recommended) - set(relevant_items))
    false_negatives = len(set(relevant_items) - set(top_recommended))
    # Calculate precision and recall
    precision = true_positives / len(top_recommended) if len(top_recommended) > 0 else 0
    recall = true_positives / len(relevant_items) if len(relevant_items) > 0 else 0
    return {
        "precision": precision,
        "recall": recall,
        "true_positives": true_positives,
        "false_positives": false_positives,
        "false_negatives": false_negatives,
    }


In [56]:
input_title = "Half-Life"
# recommended_titles = recommend(input_title)
recommended_titles = get_game_recommendations("Half-Life")

print(type(recommended_titles));

# Calculate precision and recall for the recommendation
metrics = calculate_precision_recall(input_title, recommended_titles, ground_truth, top_n=8)

print(f"Recommendations for '{input_title}':")
print(recommended_titles)
print(f"Precision: {metrics['precision']}")
print(f"Recall: {metrics['recall']}")
precision = metrics['precision']
recall = metrics['recall']

# Calculate F1 score
if precision + recall > 0:
    f1_score = 2 * (precision * recall) / (precision + recall)
else:
    f1_score = 0.0

print("Fi Score:", f1_score)



<class 'list'>
['Half-Life 2', 'Half-Life 2: Episode One', 'Half-Life 2: Episode Two', 'Portal', 'Team Fortress 2', 'Left 4 Dead', 'Portal 2']
['Half-Life 2', 'Counter-Strike: Source', 'Half-Life 2: Episode One', 'Portal', 'Half-Life 2: Episode Two', 'Team Fortress 2', 'Left 4 Dead', 'Portal 2']
Recommendations for 'Half-Life':
['Half-Life 2', 'Counter-Strike: Source', 'Half-Life 2: Episode One', 'Portal', 'Half-Life 2: Episode Two', 'Team Fortress 2', 'Left 4 Dead', 'Portal 2', 'POSTAL 2', 'Black Mesa']
Precision: 0.875
Recall: 1.0
0.9333333333333333


In [74]:
all_titles = ["Call of Duty® 2", "Half-Life 2: Episode One","Battlefield: Bad Company™ 2"]
f1_scores = []
total_tp, total_fp, total_fn = 0, 0, 0  # For micro-average
for input_title in all_titles:
    # Get recommendations
    recommended_titles = recommend(input_title)
    # Calculate precision and recall
    metrics = calculate_precision_recall(input_title, recommended_titles, ground_truth, top_n=5)
    precision = metrics['precision']
    recall = metrics['recall']
    # Calculate F1 score for this title
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0
    # Add F1 score to list for macro-average
    f1_scores.append(f1)
    # Aggregate TP, FP, FN for micro-average
    total_tp += metrics['true_positives']
    total_fp += metrics['false_positives']
    total_fn += metrics['false_negatives']
# Micro-Averaged F1
if total_tp + total_fp > 0 and total_tp + total_fn > 0:
    micro_precision = total_tp / (total_tp + total_fp)
    micro_recall = total_tp / (total_tp + total_fn)
    micro_f1 = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)
else:
    micro_f1 = 0.0
# print(f"Macro-Averaged F1: {macro_f1}")
print(f"Precision: {micro_precision}" )
print(f"Recall: {micro_recall}" )
print(f"Averaged F1 Score: {micro_f1}")


Precision: 0.8
Recall: 0.8571428571428571
Averaged F1 Score: 0.8275862068965518


['Half-Life 2',
 'Counter-Strike: Source',
 'Half-Life 2: Episode One',
 'Portal',
 'Half-Life 2: Episode Two',
 'Team Fortress 2',
 'Left 4 Dead',
 'Portal 2',
 'POSTAL 2',
 'Black Mesa']